In [48]:
# Essential plotting and scientific libraries
import matplotlib.pyplot as pl
from matplotlib.colors import LinearSegmentedColormap
%matplotlib inline
import numpy as np

from numpy import fft

from skimage.io import imread
from skimage.transform import resize

# IPython display imports
from ipywidgets import interact, FloatSlider, IntSlider, RadioButtons, Button, HBox, VBox
from IPython.display import Image, display, clear_output


from scipy.misc import face
face = face() # An example image we can use
from scipy.signal import convolve2d

import wave

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

HTML('''<script>
    code_show=true; 
    function code_toggle() {
         if (code_show){
         $('div.input').hide();
         } else {
         $('div.input').show();
         }
         code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>
    ''')

## Describing Imaging Systems
<div style="float: left; width: 45%;">
<br>
<ul>
<li> No imaging system is perfect!  <br><br>
<li> Images only provide a reasonable approximation of the underlying object we are trying to capture. <br><br>
<li> For example, medical images typically provide some information about a particular biological property of the tissue at a given position in space, $(x, y)$ <br><br>
</ul>
</div>
<div style="float: right; width: 50%;">
 <img src='images/medical_image_examples.jpg' width = '80%'>
</div>

## Describing Imaging Systems
<div style="float: left; width: 45%;">
<br>
<ul>
    <li> In the following slides we introduce a general formulation for characterising imaging systems.<br><br>
    <li> We deonte the <b>object</b> as $f(\alpha, \beta)$ , and the <b>image</b> as $g(x, y)$<sup>1</sup><br><br>
    <li> We also typically assume that $f(\alpha, \beta) \in \mathbb{R}^{+}$ and $g(x, y) \in \mathbb{R}^{+}$<br><br>
    <li> In its most general sense, the imaging system can be thought of as an operator, $\mathcal{H}$, operating on the opbject to produce the image.<br><br>
</ul>
<hr></hr>
<sup>1. Here we assume we are working with 2D images, but these methods are applicable to 3 or more dimensions as well!</sup>
</div>
<br><br><br>
<div style="float: right; width: 50%;">
<img src='images/image_object.jpg' width = '90%'>
</div>

## Linear Imaging Systems
<ul>
    <li>Consider now that the same imaging system is used to capture images, $g_{1}(x, y)$ and $g_{2}(x, y)$, of two objects, $f_{1}(\alpha, \beta)$ and $f_{2}(\alpha, \beta)$ respectively.<br><br>
    <li> A <b>linear imaging system</b> is one that satisfies the following property:<br><br>
\begin{align}
\mathcal{H}\left(g_{1} + g_{2}\right) &= \mathcal{H}\left(g_{1}\right) + \mathcal{H}\left(g_{2}\right) \\ 
        &= f_{1} + f_{2}
\end{align}<br>
     <li> For example, in PET imaging, if the concentration of radioactive tracer doubles, then so does the intensity on the final image.  Another example could be doubling of the exposure time when taking a picture (although there will be a limit!)<br><br>
     <li> In practise nearly all imaging systems are considered to be linear.
</ul>

## Point-Spread Function
<div style="float: left; width: 50%;">
<br>
<ul>
    <li> Lets go back to our example of the pin-hole camera with a finite aperture.<br><br>
    <li> What happens if we use a point-source at position $(\alpha, \beta)$ as the object: $f(\alpha', \beta') = \delta(\alpha' - \alpha, \beta' - \beta)$?<br><br>
    <li> We will end up with the <b>point-spread function</b> (PSF) describing the imaging-system for the location of the point source, $h(x, y; \alpha, \beta)$.<br><br>
    <li> The final image will be the sum (integral) of the contribution from all point-sources that make up the object (assuming the imaging system is linear):
    $$
    g(x, y) = \iint\limits_{-\infty}^{\infty} h(x, y; \alpha, \beta)\cdot f(\alpha, \beta)\text{d}\alpha\text{d}\beta
    $$
</ul>
</div>
<br><br><br>
<div style="float: right; width: 40%;">
    <img src='images/pinhole_projection.jpg' width = '90%'>
</div>

## Shift Invariance
<div style="float: left; width: 40%;">
<br>
<ul>
    <li>If the PSF is <b>shift invariant</b> then $h$ will not change for different positions in the source frame-of-reference.<br><br>
    <li> Instead, the final image will be a <b>convolution</b> of the shift-invariant PSF with the object:<br><br>
        \begin{align}
            g(x, y) &= \iint\limits_{-\infty}^{\infty} h(x-\alpha, y-\beta)\cdot f(\alpha, \beta)\text{d}\alpha\text{d}\beta            
        \end{align}
        <br>
    <li> In practice nearly all imaging systems are considered to be shift invariant (even if it is know they are not!).
</ul>
</div>
<div style="float: right; width: 50%;">
    <img src='images/PSF_examples.jpg' width = '90%'>
</div>

## Example of the PSF
This example taken from earlier on works as a good example of the (shift-invariant) PSF in action.  Using a Guassian distribution as the PSF and increasing the variance of the distribution appears to make the image more blurry!

In [47]:
# Convert the red-channel of the 'face' to a floating point array (value 0 to 1):
face_R = face[:, :, 0].astype('float32')/255

# Make it smaller to speed thing up a little....
face_R = resize(face_R, (face_R.shape[0] / 5, face_R.shape[1] / 5))

PSF_variance_slider = FloatSlider(min=0.0, max=30, step=0.5, value=0, continuous_update=False)
PSF_variance_slider.add_class("mytext")
    
# decorate the plot function with an environment from the UIs:
@interact(variance=PSF_variance_slider)
def convolute_PSF(variance):
    
    # We can't actually use zero here so just make very small.
    if variance == 0.0:
        variance = 1e-10
    
    # convolve with a disk of varying radius
    x = np.linspace(-20, 20, 41)
    X, Y = np.meshgrid(x, x)
    
    gaussian = (1.0 / (2*np.pi*np.sqrt(variance))) * np.exp(-0.5 * (X**2 + Y**2) / variance)
    
    f = pl.figure(figsize = (25, 6))
    ax1 = f.add_subplot(131) 
    ax1.imshow(gaussian / gaussian.max(), cmap = 'gray', clim = (0, 1), interpolation = 'None')
    ax1.set_title(r'$PSF, h(x - \alpha, y - \beta)$', fontsize = 20.0)
    
    ax2 = f.add_subplot(132)
    ax2.imshow(face_R, cmap = 'gray', interpolation = 'None')
    ax2.set_title(r'$f(\alpha, \beta)$', fontsize = 20.0)
    
    ax3 = f.add_subplot(133)
    ax3.imshow(convolve2d(face_R, gaussian, mode='same'), cmap = 'gray', interpolation = 'None')
    ax3.set_title(r'$g(x, y)$', fontsize = 20.0)
    
    pl.show()

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description=u'variance', max=30.0, step=0.5, _dom_classes=('mytext',)), Output()), _dom_classes=('widget-interact',))

## Transfer Functions
<ul>
    <li> Recall the convoilution theorem, which states that the Fourier transform of the convolution between two signals, $f$ and $h$, is equal to the product of the fourier trannform of each signal, $F$ and $H$ respectively. <br><br>
    <li> In two-dimensions this means that we have:<br><br>
    \begin{align}
       \text{FT}\left\{ g(x, y) \right\} &= \text{FT}\left\{\iint\limits_{-\infty}^{\infty}h(x-\alpha, y-\beta)\cdot f(\alpha, \beta)\text{d}\alpha \text{d}\beta \right\}\\\\
        &= \text{FT}\left\{h(x, y)\right\} \cdot \text{FT}\left\{g(x, y)\right\} \\\\
        \therefore G(k_{x}, k_{y}) &= H(k_{x}, k_{y})\cdot F(k_{x}, k_{y})
    \end{align}<br>
    <li> $H(k_{x}, k_{y})$, the Fourier transform of the point-spread function, is called the <b>Optical Transfer Function</b> (OTF) <br><br>
    <li> In general, the OTF is complex valued and so it can be simpler to define the <b>Modulation Transfer Function</b> (MTF) as the absolute value:<br><br>
        \begin{align}
            \text{MTF} &= |\text{OTF}| = |H(k_{x}, k_{y})|
        \end{align}
</ul>

## Transfer Functions - what do they represent?
<div style="float: left; width: 50%;">
<br>
<ul>
    <li> $F$ describes the frequencies present in the <i>object</i> and $G$ the frequencies in the <i>image</i><br><br>
    <li> If the magnitude of $H$ (i.e. the MTF) is low, then these frequencies in object are attenuated in the final image (i.e. modulations in the object at these frequencies are not <i>transferred</i> into the image!)
</ul>
</div>
<br><br>
<div style="float: right; width: 40%; border:1px solid black;">
<br>
\begin{gather*}
\text{image frequencies} = \text{OTF} \cdot \text{object frequencies} \\\\
G(k_{x}, k_{y}) = H(k_{x}, k_{y})\cdot F(k_{x}, k_{y}) \\\\
\text{MTF} = |H(k_{x}, k_{y})|
\end{gather*}
<br>
</div>

In [80]:
# Convert the red-channel of the 'face' to a floating point array (value 0 to 1):
face_R = face[:, :, 0].astype('float32')/255

# Make it smaller to speed thing up a little....
face_R = resize(face_R, (face_R.shape[0] / 5, face_R.shape[1] / 5))

PSF_width_slider = FloatSlider(min=0.0, max=30, step=0.5, value=0, continuous_update=False)
    
# decorate the plot function with an environment from the UIs:
@interact(width=PSF_width_slider)
def compute_MTF(width):
    
    # We can't actually use zero here so just make very small.
    if width == 0.0:
        width = 1e-10
    
    # convolve with a disk of varying radius
    x = np.linspace(-20, 20, 41)
    X, Y = np.meshgrid(x, x)
    
    PSF = 1.0 * np.logical_and(np.abs(X) <= width / 2, np.abs(Y) <= width / 2)
    PSF_1D = 1.0 * (np.abs(x) <= width / 2)
    kx = fft.fftshift(fft.fftfreq(len(x), 40))
    MTF = np.abs(fft.ifftshift(fft.fft(fft.fftshift(PSF_1D))))
    MTF = MTF/MTF[21]
    
    image = convolve2d(face_R, PSF, mode='same')
    
    f = pl.figure(figsize = (12, 5))
    ax1 = f.add_subplot(231) 
    ax1.imshow(face_R, cmap = 'gray', interpolation = 'None')
    ax1.set_title(r'Object $f(\alpha, \beta)$', fontsize = 20.0)
    ax1.set_xticks([]), ax1.set_yticks([])
    
    ax2 = f.add_subplot(232) 
    ax2.plot(PSF_1D, 'k-')
    ax2.set_title(r'PSF $h(\alpha)$', fontsize = 20.0)
    ax2.set_xticks([]), ax2.set_yticks([])
    
    ax3 = f.add_subplot(233)
    ax3.imshow(image, cmap = 'gray', interpolation = 'None')
    ax3.set_title(r'Image $g(x, y)$', fontsize = 20.0)
    ax3.set_xticks([]), ax3.set_yticks([])
    
    
    row_object = face_R[75, :]
    row_object_fft = fft.ifftshift(fft.fft(fft.fftshift(row_object)))
    
    row_image = image[75, :]
    row_image_fft = fft.ifftshift(fft.fft(fft.fftshift(row_image)))
    
    ax4 = f.add_subplot(234) 
    ax4.semilogy(np.abs(row_object_fft/row_object_fft.max()), 'k-')
    ax4.set_title(r'$F(k_{x})$', fontsize = 20.0)
    ax4.set_xticks([])
    
    ax5 = f.add_subplot(235) 
    ax5.semilogy(MTF, 'k-')
    ax5.set_title(r'MTF $H(k_{x})$', fontsize = 20.0)
    ax5.set_xticks([])
    
    ax6 = f.add_subplot(236) 
    ax6.semilogy(np.abs(row_image_fft/row_image_fft.max()), 'k-')
    ax6.set_title(r'$G(k_{x})$', fontsize = 20.0)
    ax6.set_xticks([])
    
    pl.tight_layout()
    
    pl.show()

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description=u'width', max=30.0, step=0.5), Output()), _dom_classes=('widget-interact',))